# Séance nº5 : régression (fin) ; intégration numérique

In [3]:
import numpy as np 
import matplotlib.pyplot as plt #import des packages

[Years, SMB, Albedo_mean]=np.loadtxt('Donnees_BM-Albedo_Argentiere_2000-2014.csv',skiprows=1,unpack=True)
#import des données "unpack" permet de séparer nos données en 3 variables dans notre cas 

mean_x=np.mean(Albedo_mean)
n=len(Albedo_mean)
Var_albedo=1/n*np.sum((Albedo_mean-mean_x)**2) #Variance albedo
std_x=np.sqrt(Var_albedo)#ecart type albedo

mean_y=np.mean(SMB)
Var_SMB=1/n*np.sum((SMB-mean_y)**2)#Variance Bilan de masse
std_y=np.sqrt(Var_SMB)#ecart type Bilan de masse

#coefficient de corrélation linéaire
r = 1./(std_x * std_y * n) * np.sum(  (Albedo_mean-mean_x) * (SMB-mean_y))
#coefficient de détermination (google it!!!)
r2 = r**2

# Puis, on peut estimer, a et b, respectivement la pente et l'intercept de la droite de régression 
#ayant pour équation bm = a*alb+b.

a = r * std_y / std_x #calcul de a. la pente de notre regression linéaire
b = np.mean(SMB) - a * np.mean(Albedo_mean) #calcul de b. l'intercept ou l'ordonnée à l'origine de notre régression 

sse = np.sum( (SMB - (a * Albedo_mean + b))**2 )#somme au carré des erreurs par rapport au données mesurées
std_e = np.sqrt(( 1./ (n - 2)) * sse ) 
std_a = std_e / (np.sqrt(n) * std_x) # Ecart-type sur la pente = 3.2
std_b = std_a * np.sqrt((1./n) * np.sum(Albedo_mean**2)) # Ecart-type sur l'intercept = 1.3


# Puis on trace cette droite

regression = a*Albedo_mean + b # Ici on recalcule les valeurs de bilan si elles étaient toutes sur la droite 
#(cela nous permet en fait de tracer cette droite de correlation.)


alea=np.random.randint(0,len(SMB)-1,size=(15,3000))# je crée une matrice de 15 par 3000 contenant des valeurs 
#aléatoires comprises entre 0 et 14. 

bilan=SMB[alea] #je crée un matrice de même taille que "alea" mais contenant les valeurs de SMB correspondant
# à la valeur de alea. 
albedo_m=Albedo_mean[alea] #idem pour albedo



## 1. Fin de l’exercice sur la corrélation entre albedo et bilan de masse d’un glacier

Le bilan de masse (gain ou perte nette de masse de l’ensemble du glacier) a aussi été mesuré. Le jeu de données (fichier ‘Donnees\_BM-Albedo\_Argentiere\_2000-2014.csv’) contient l’albedo moyen (minimum atteint en septembre) et le bilan de masse, pour 15 années.

\* Estimation des incertitudes du modèle linéaire par approche Monte-Carlo : ce modèle est appliqué sur 3000
échantillons des données (technique 'Bootstrap'). Ré-échantillonner les couples de valeurs albedo - bilan de
masse 3000 fois (avec remise). Pour chaque échantillon (parmi les 3000) de 15 couples de valeurs, calculer
pente et intercept de la régression linéaire. Calculer enfin moyenne et écart type de ces deux paramètres.
Comparer avec les statistiques obtenues ci-dessus sous hypothèse de distribution normale.

In [4]:
alea=np.random.randint(0,len(SMB)-1,size=(15,3000))# je crée une matrice de 15 par 3000 contenant des valeurs 
#aléatoires comprises entre 0 et 14. 

bilan=SMB[alea] #je crée un matrice de même taille que "alea" mais contenant les valeurs de SMB correspondant
# à la valeur de alea. 
albedo_m=Albedo_mean[alea] #idem pour albedo


## 2. Introduction à l’intégration numérique

 Il est courant en géosciences de faire face à une intégrale qu’il est impossible de calculer analytiquement. C’est souvent pour une des deux raisons suivantes :

on connaît l’équation différentielle qui décrit le phénomène (lois physiques), mais pas sa primitive ;

on connaît le phénomène par des mesures de terrain, et on a besoin d’intégrer l’un des paramètres dans le temps ou l’espace (intégration d’un débit ou des précipitations, par exemple).

 Pour intégrer numériquement une équation différentielle, ou des données, on a besoin des valeurs pour des pas de temps ou d’espace ‘discrets’. Entre ces pas, les valeurs exactes de ces paramètres ne sont pas connues, et des hypothèses sur ces valeurs manquantes sont nécessaires. Cette discrétisation, inhérente à l’approche numérique, introduit une erreur sur le résultat de l’intégration. Pour réduire cette erreur, il faut d’une part réduire la taille des pas (mais au détriment du coût de calcul), d’autre part utiliser des hypothèses sur les valeurs manquantes plus ‘réalistes’ (ce qu’on appelle des schémas numériques).

### 2. 1 *Le schéma d’intégration : la méthode des trapèzes*

 Soit une fonction *f*(x), que l’on veut intégrer sur l’intervalle \[x<sub>0</sub>,x<sub>N</sub>\]. Cet intervalle est discrétisé en N intervalles égaux de longueur ∆x. La méthode d’intégration des trapèzes consiste à approcher l’intégrale de la fonction *f* sur chaque intervalle \[x<sub>i\\ </sub>, x<sub>i+1</sub>\] = ∆x par l’aire d’un trapèze :

$\int_{x_i}^{x_{i+1}}f(x) dx \approx \frac{\Delta x (f(x_i)+f(x_{i+1}))}{2} $(1)

Cette formule (1) est le résultat de l’intégration du développement limité d’ordre 1 de *f*(x), dont le résultat exact est :

$\int_{x_i}^{x_{i+1}}f(x) dx=\frac{\Delta x (f(x_i)+f(x_{i+1}))}{2} +O(\Delta x^3 f'')$ (2)

Le second terme du second membre de l’équation (2) quantifie l’erreur commise sur l’intégrale de *f* approchée par la méthode des trapèzes. En choisissant ce schéma, l’ordre de grandeur de l’erreur est quantifié par la valeur faisant intervenir la valeur du pas d’intégration ∆x ainsi que la dérivée seconde de la fonction *f*. (La valeur de l’erreur est fondamentale quand on cherche à faire des calculs précis et justes.)

L’interprétation “graphique” du schéma numérique est donnée sur la figure. On comprend alors que plus le pas d’intégration ∆x est petit et plus on se rapproche d’un calcul précis pour la valeur finale de l’intégrale.
![figure2](../TP4/figureTP4_2.png)
Figure : a) Fonction f(x) à intégrer sur l’intervalle d’intégration \[x<sub>0</sub>,x<sub>N</sub>\] , discrétisé en (N+1) points équidistants. Les figures b) et c) illustrent le schéma numérique. L’intégrale entre deux points x<sub>i</sub> et x<sub>i+1</sub> de f(x) est approchée en calcul numérique soit par b) l’aire du trapèze x<sub>i</sub> x<sub>i+1</sub> f<sub>i+1</sub> f<sub>i</sub> , soit par c) l’aire du rectangle de base x<sub>i</sub> et x<sub>i+1</sub> et de hauteur (f<sub>i</sub> + f<sub>i+1</sub> )/2.

Soit une fonction *f* que l’on veut intégrer sur un intervalle \[x<sub>0</sub>,x<sub>N</sub>\] par la méthode des trapèzes.

#### a) Ecrire la relation entre ∆x et N (le nombre d’intervalles).

#### b) Montrer que l’aire du trapèze et l’aire du rectangle des figures b et c sont les mêmes.

#### c) Ecrire la relation (1) sur tout l’intervalle \[x<sub>0</sub>,x<sub>N</sub>\], en fonction des f(x<sub>i</sub>), i∈\[0,N\].

### 2.2 Intégration de la fonction *f*(x) = x<sup>2</sup>.sinx sur l’intervalle \[0,π\]

#### a) Intégration analytique (par parties).

#### b) Ecrire l’algorithme permettant d’appliquer le schéma numérique (1) pour l’intégration de *f*.

Au début de l’algorithme, l’utilisateur donne le nombre de pas (N). On utilisera une itération explicite (boucle). La fonction *f* est calculée dans une structure fonction.

#### c) Transcrire l’algorithme en programme. Calculer l’intégrale de *f* pour des valeurs différentes de N (10, 100, 1000). Calculer l’erreur. Calculer l’intégrale avec une fonction résidente de type *integrate* (voir la syntaxe).